In [1]:
# pip install pandas yfinance pyarrow tqdm
import pandas as pd, yfinance as yf, textwrap, re, math, requests, time
from pathlib import Path
from tqdm import tqdm


Parses 500 most weighted tickers in the iShares MSCI World ETF

In [19]:
csv_file = "../data/archive/URTH_holdings.csv"  # Fund Holdings as of,""Jul 25, 2025"""
raw = pd.read_csv(csv_file)
tickers = (
    raw["Ticker"]
    .dropna()   # only drops National Bank of Canada with "NaN" Ticker
    .head(500)  # only considers 500 most weighted companies
    .tolist()
)

# (optional) manual fixes for non‑Yahoo tickers
# mapping = {"ADS": "ADS.DE", "NESN": "NESN.SW", ...}
# tickers = [mapping.get(t, t) for t in tickers]

yfinance requires exchange suffixes for non-american tickers, e.g., NOVN has Yahoo symbol NOVN.SW where .SW represents the SIX exchange in Switzerland

In [18]:
# Find strings containing spaces
strings_with_spaces = [t for t in tickers if ' ' in t]

# Check if any were found
if strings_with_spaces:
    print("Found strings with spaces:", strings_with_spaces)
else:
    print("No strings with spaces found")


Found strings with spaces: ['NOVO B', 'INVE B', 'VOLV B', 'NDA FI', 'ATCO A', 'ASSA B']


In [26]:
def resolve_yf_symbol(raw: str, retries: int = 3):
    """
    Return a valid Yahoo Finance ticker for `raw`
    using Yahoo’s public search endpoint.
    """
    # # quick patch for common manual errors -----------------------------
    # manual = {
    #     "BRKB": "BRK-B", "NOVO B": "NOVO-B.CO",
    #     "BP.": "BP.L", "RR.": "RR.L"
    # }
    # if raw in manual:
    #     return manual[raw]

    # exchanges " " with "-"
    raw = raw.replace(" ", "-")

    # checks the stock exchange of tickers
    url = "https://query2.finance.yahoo.com/v1/finance/search"
    for _ in range(retries):
        r = requests.get(url, params={"q": raw, "quotes_count": 1, "news_count": 0},
                         headers={"User-Agent": "Mozilla/5.0"})
        if r.ok and r.json().get("quotes"):
            return r.json()["quotes"][0]["symbol"]
        time.sleep(0.5)          # back‑off on errorsx
    return None                  # couldn’t resolve

tickers_raw = ['ATCO A', 'LSEG', 'EOAN', 'WBC', 'ENEL', '8031', 'HOLN', '6861', '7267', 'LONN', 'D05', 'O39', '6857', 'ENGI', 'VOLV B', '8058', 'CPG', 'CSU', '9433', '8411', 'BARC', 'LLOY', 'EXPN', '4502', 'GLEN', '6503', '8001', 'NG.', '388', '9983', 'NDA FI', 'BA.', '4063', 'XTSLA', 'DB1', '9434', '8766', 'INVE B', '7741', '4568', 'ADYEN', 'BAS']           # your original list
tickers_fixed = [resolve_yf_symbol(t) or t for t in tickers_raw]
# tickers_fixed = [t for t in tickers_fixed if t is not None]  # drop None’s


In [ ]:
['ATCO-A.ST', 'LSEG.L', 'EOAN.DE', 'WBCPH.AX', 'ENIC', '8031.T', 'HOLN.SW', '6861.T', '7267.T', 'LONN.SW', 'D05.SI', 'O39.SI', '6857.T', '047040.KS', 'VOLV-B.ST', '8058.T', '0P0001JNFN', 'CSU.TO', '9433.T', '8411.HK', 'BCS', 'LLOY.L', 'EXPN.L', '4502.T', 'GLEN.L', '6503.T', '8001.T', 'NGNISX=X', '3882.HK', '9983.HK', 'NDA-FI.HE', 'BAMRSD=X', '4063.T', 'XTSLA', 'DB1.DE', '9434P.T', '8766.T', 'INVE-B.ST', '7741.T', '4568.TWO', 'ADYEY', 'NGO=F']

In [27]:
data = yf.download(tickers_fixed, group_by="ticker", threads=False, multi_level_index=True)
data.xs('Close', level=1, axis=1)

C:\Users\strakad\AppData\Local\Temp\ipykernel_29508\29372600.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers_fixed, group_by="ticker", threads=False, multi_level_index=True)
[*********************100%***********************]  42 of 42 completed

2 Failed downloads:
['9434P.T']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo)')
['XTSLA']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")')


Ticker,3882.HK,8001.T,6857.T,GLEN.L,LSEG.L,8031.T,ATCO-A.ST,9983.HK,BAMRSD=X,CSU.TO,...,NGO=F,047040.KS,D05.SI,LLOY.L,ADYEY,6503.T,INVE-B.ST,HOLN.SW,NDA-FI.HE,NGNISX=X
Date,,,,,,,,,,,,,,,,,,,,,
2025-06-30,1.16,7556.0,10655.0,283.600006,10635.000000,2947.0,152.850006,1.10,NaN,4993.200195,...,NaN,4180.0,44.910000,76.699997,18.309999,3111.0,279.750000,58.900002,12.610,NaN
2025-07-01,NaN,7523.0,10765.0,291.250000,10650.000000,2925.0,153.250000,NaN,NaN,NaN,...,NaN,4225.0,45.110001,76.360001,18.020000,3142.0,279.799988,58.400002,12.435,NaN
2025-07-02,1.15,7605.0,10425.0,306.000000,10805.000000,2957.0,157.350006,1.08,NaN,4973.240234,...,NaN,4160.0,44.849998,73.540001,18.010000,3081.0,281.649994,59.520000,12.510,NaN
2025-07-03,1.13,7592.0,10530.0,306.200012,10785.000000,3015.0,157.899994,1.10,NaN,5030.089844,...,NaN,4180.0,44.950001,75.879997,18.000000,3056.0,283.399994,59.680000,12.685,NaN
2025-07-04,1.15,7537.0,10675.0,302.850006,10700.000000,2997.5,156.800003,1.12,NaN,5005.000000,...,NaN,4095.0,45.209999,75.360001,NaN,3085.0,281.299988,59.360001,12.520,NaN
2025-07-07,1.12,7575.0,10685.0,297.899994,10800.000000,2996.0,157.100006,1.14,NaN,4998.970215,...,NaN,4035.0,45.669998,75.680000,18.170000,3056.0,282.000000,60.740002,12.665,NaN
2025-07-08,1.13,7619.0,10955.0,306.399994,10870.000000,3026.0,158.000000,1.13,NaN,4972.020020,...,NaN,4100.0,45.730000,75.660004,18.340000,3066.0,284.149994,61.459999,12.635,NaN
2025-07-09,1.11,7663.0,10980.0,298.200012,10840.000000,3064.0,160.000000,1.12,NaN,4956.279785,...,NaN,4175.0,45.650002,75.559998,18.480000,3116.0,286.200012,62.560001,12.935,NaN
2025-07-10,1.11,7590.0,11245.0,309.950012,10880.000000,3018.0,163.449997,1.14,NaN,4955.589844,...,NaN,4155.0,45.820000,76.040001,17.690001,3096.0,293.600006,63.560001,12.960,NaN


In [20]:
data = yf.download(tickers, group_by="ticker", threads=False, multi_level_index=True)
data.xs('Close', level=1, axis=1)

C:\Users\strakad\AppData\Local\Temp\ipykernel_29508\822265226.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, group_by="ticker", threads=False, multi_level_index=True)
[*********************100%***********************]  495 of 495 completed

122 Failed downloads:
['ATCO A', 'EOAN', '6098', 'PGHN', '3382', '4519', 'LONN', 'UMG', 'CS', 'U11', '9432', 'CSU', '6503', 'GLEN', '388', 'NDA FI', 'NESN', '9984', '4063', 'ULVR', 'DB1', 'SIKA', '7741', 'RR.', '4568', 'HSBA', 'BAS', '8750', 'LSEG', '8002', '7974', '7203', '8031', '1299', '6861', '6702', 'ASSA B', 'CPG', 'LLOY', '4502', 'NG.', '9983', 'ZURN', '9434', '8725', 'MUV2', 'ADYEN', 'BRKB', '8316', '8306', 'BAYN', 'BP.', 'NOVO B', '7011', '6301', 'O39', '7267', 'ADS', 'VOLV B', '8058', '8411', '8035', 'EXPN', 'BA.', 'XTSLA', 'ABBN', '8766', 'INVE B', 'WBC', 'ENEL', 'NOVN', 'HOLN', 'D05', '6367', '6857', 'ENGI', '9433', 'UBSG', '6501', 'BARC', '8001', '2914', '6758', 'CBK', '

Ticker,MCO,ACN,JNJ,ATCO A,GOOGL,FTNT,TEL,EOAN,PCG,APP,...,CNQ,CNR,CBK,AXP,MRK,6701,SRE,SIE,ETN,DGE
Date,,,,,,,,,,,,,,,,,,,,,
2025-06-30,501.589996,298.890015,152.750000,NaN,176.229996,105.720001,168.669998,NaN,13.94,350.079987,...,31.400000,69.739998,NaN,318.176697,79.160004,NaN,75.769997,NaN,356.989990,NaN
2025-07-01,502.480011,302.619995,155.919998,NaN,175.839996,102.430000,170.270004,NaN,14.13,336.690002,...,31.660000,65.269997,NaN,321.717743,81.809998,NaN,76.180000,NaN,355.040009,NaN
2025-07-02,497.119995,302.290009,155.559998,NaN,178.639999,102.209999,173.410004,NaN,13.99,336.000000,...,32.410000,73.250000,NaN,324.789978,82.389999,NaN,74.820000,NaN,358.190002,NaN
2025-07-03,505.059998,304.779999,156.009995,NaN,179.529999,105.660004,174.289993,NaN,13.91,341.640015,...,32.220001,72.779999,NaN,328.130005,80.930000,NaN,75.120003,NaN,362.220001,NaN
2025-07-07,502.369995,300.540009,155.270004,NaN,176.789993,106.650002,171.139999,NaN,13.64,345.000000,...,31.750000,70.750000,NaN,322.730011,80.900002,NaN,74.389999,NaN,358.489990,NaN
2025-07-08,499.019989,303.329987,155.789993,NaN,174.360001,107.540001,172.429993,NaN,13.67,344.750000,...,32.130001,69.470001,NaN,316.980011,81.370003,NaN,74.320000,NaN,356.980011,NaN
2025-07-09,502.220001,297.399994,156.279999,NaN,176.619995,107.650002,172.649994,NaN,13.52,352.739990,...,31.980000,69.080002,NaN,317.350006,83.709999,NaN,74.489998,NaN,359.779999,NaN
2025-07-10,505.720001,288.359985,157.690002,NaN,177.619995,100.199997,175.029999,NaN,13.54,346.320007,...,31.790001,72.389999,NaN,325.239990,84.019997,NaN,74.900002,NaN,357.640015,NaN
2025-07-11,499.529999,281.059998,156.899994,NaN,180.190002,99.059998,177.110001,NaN,13.42,335.100006,...,32.099998,75.599998,NaN,319.470001,83.360001,NaN,74.559998,NaN,360.619995,NaN


In [21]:
set(['NOVO B', 'INVE B', 'VOLV B', 'NDA FI', 'ATCO A', 'ASSA B']).issubset(set(['ATCO A', 'EOAN', '6098', 'PGHN', '3382', '4519', 'LONN', 'UMG', 'CS', 'U11', '9432', 'CSU', '6503', 'GLEN', '388', 'NDA FI', 'NESN', '9984', '4063', 'ULVR', 'DB1', 'SIKA', '7741', 'RR.', '4568', 'HSBA', 'BAS', '8750', 'LSEG', '8002', '7974', '7203', '8031', '1299', '6861', '6702', 'ASSA B', 'CPG', 'LLOY', '4502', 'NG.', '9983', 'ZURN', '9434', '8725', 'MUV2', 'ADYEN', 'BRKB', '8316', '8306', 'BAYN', 'BP.', 'NOVO B', '7011', '6301', 'O39', '7267', 'ADS', 'VOLV B', '8058', '8411', '8035', 'EXPN', 'BA.', 'XTSLA', 'ABBN', '8766', 'INVE B', 'WBC', 'ENEL', 'NOVN', 'HOLN', 'D05', '6367', '6857', 'ENGI', '9433', 'UBSG', '6501', 'BARC', '8001', '2914', '6758', 'CBK', '6701']))


True

In [13]:
# -------- 2. Batch‑download
start = "1980-01-01"
parquet_path = Path("msci_world_daily.parquet")
bsize = 200
frames = []

for batch in tqdm(range(math.ceil(len(tickers)/bsize))):
    block = tickers[batch*bsize : (batch+1)*bsize]
    data = yf.download(block, start=start, group_by="ticker", progress=False)   # progress bar hidden,
    frames.append(data.xs('Close', level=1, axis=1))    # keep just Close prices to save space


  0%|          | 0/3 [00:00<?, ?it/s]C:\Users\strakad\AppData\Local\Temp\ipykernel_29508\3455574501.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(block, start=start, group_by="ticker", progress=False)   # progress bar hidden,
HTTP Error 404: 
HTTP Error 404: 

25 Failed downloads:
['8316', 'NOVO B', '6098', '7011', 'BP.', '8306', 'NOVN', '7974', '7203', '1299', 'CS', 'UBSG', '8035', '6501', 'ZURN', 'NESN', '9984', 'ULVR', 'ABBN', '6758', 'RR.', 'MUV2', 'HSBA', 'BRKB']: YFTzMissingError('possibly delisted; no timezone found')
['BATS']: YFPricesMissingError('possibly delisted; no price data found  (1d 1980-01-01 -> 2025-07-30)')
 33%|███▎      | 1/3 [00:21<00:42, 21.13s/it]C:\Users\strakad\AppData\Local\Temp\ipykernel_29508\3455574501.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(block, start=start, group_by="ticker", progress=False)   # progress bar hidden,

43 Failed do

In [ ]:
prices = pd.concat(frames, axis=1).sort_index()
prices.to_parquet(parquet_path, compression="zstd")

In [ ]:
# Let's inspect what data we get from yfinance
sample_data = yf.download(tickers[0], start=start, auto_adjust=True)
print("Sample data columns:", sample_data.columns.tolist())


In [ ]:
# -------- 3. Quick sanity snapshot
snapshot = prices.tail(1).T.reset_index()
snapshot.columns = ["Ticker", "Last Close"]
snapshot.to_csv("snapshot_latest.csv", index=False)

print("Saved", parquet_path, "and snapshot_latest.csv")